In [ ]:
import os
from PIL import Image
import numpy as np
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def print_model_size(model):
    """ Print the size of the model.
    
    Args:
        model: model whose size needs to be determined

    """
    torch.save(model.state_dict(), "temp.p")
    print('Size of the model(MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')
def numToLong(input):
    return {"0": "Cassava Bacterial Blight (CBB)", 
            "1": "Cassava Brown Streak Disease (CBSD)", 
            "2": "Cassava Green Mottle (CGM)", 
            "3": "Cassava Mosaic Disease (CMD)", 
            "4": "Healthy"}[str(input)]
np.random.seed(42)
df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
df['long_labels'] = df.label.map(lambda x: numToLong(x))
df.shape

In [ ]:
remove_amounts = [x-2577//4 if x!=2577 else 0 for x in list(df.long_labels.value_counts())]
rm_dict = {0:remove_amounts[4],1:remove_amounts[2],2:remove_amounts[3],3:remove_amounts[0]}

df_pruned = df.copy() 
# for i in [0,1,2,3]:
#     frac = rm_dict[i]/df_pruned[df_pruned['label']==i].count()[0]
#     print("removing ", df_pruned[df_pruned['label']==i].count()[0], numToLong(i))
#     df_pruned = df_pruned.drop(df_pruned[df_pruned['label'] == i].sample(frac=frac).index)
# df_pruned.long_labels.value_counts().plot.bar()
def is_healthy(number):
    if number==4:
        return 1 #healthy
    else:
        return 0 #is_sick
df_pruned['is_healthy'] = df_pruned['label'].apply(lambda x: is_healthy(x))
df_pruned['is_healthy']
df=df_pruned[['image_id','is_healthy']]
df2=df_pruned[['image_id','long_labels']]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
%matplotlib inline  
from sklearn.model_selection import StratifiedKFold
from joblib import load, dump
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from fastai import *
import fastai
from fastai.vision import *
from fastai.vision.data import ImageDataLoaders
from fastai.vision.augment import aug_transforms
from fastai.callback.tensorboard import TensorBoardCallback
from fastai.vision.all import Resize, RandomSubsetSplitter, aug_transforms, cnn_learner
from torchvision import models as md
# from efficientnet_pytorch import EfficientNet
from pathlib import Path
from torchvision.utils import make_grid
import os
SZ = 224

In [ ]:
import torch
from torchvision.models.resnet import Bottleneck, BasicBlock, ResNet, model_urls
import torch.nn as nn
from torchvision.models.utils import load_state_dict_from_url
from torch.quantization import QuantStub, DeQuantStub, fuse_modules
from torch._jit_internal import Optional

import torch
from torch import nn


def _replace_relu(module):
    reassign = {}
    for name, mod in module.named_children():
        _replace_relu(mod)
        # Checking for explicit type instead of instance
        # as we only want to replace modules of the exact type
        # not inherited classes
        if type(mod) == nn.ReLU or type(mod) == nn.ReLU6:
            reassign[name] = nn.ReLU(inplace=False)

    for key, value in reassign.items():
        module._modules[key] = value


def quantize_model(model, backend):
    _dummy_input_data = torch.rand(1, 3, 299, 299)
    if backend not in torch.backends.quantized.supported_engines:
        raise RuntimeError("Quantized backend not supported ")
    torch.backends.quantized.engine = backend
    model.eval()
    # Make sure that weight qconfig matches that of the serialized models
    if backend == 'fbgemm':
        model.qconfig = torch.quantization.QConfig(
            activation=torch.quantization.default_observer,
            weight=torch.quantization.default_per_channel_weight_observer)
    elif backend == 'qnnpack':
        model.qconfig = torch.quantization.QConfig(
            activation=torch.quantization.default_observer,
            weight=torch.quantization.default_weight_observer)

    model.fuse_model()
    torch.quantization.prepare(model, inplace=True)
    model(_dummy_input_data)
    torch.quantization.convert(model, inplace=True)

    return

__all__ = ['QuantizableResNet', 'resnet18', 'resnet50',
           'resnext101_32x8d']


quant_model_urls = {
    'resnet18_fbgemm':
        'https://download.pytorch.org/models/quantized/resnet18_fbgemm_16fa66dd.pth',
    'resnet50_fbgemm':
        'https://download.pytorch.org/models/quantized/resnet50_fbgemm_bf931d71.pth',
    'resnext101_32x8d_fbgemm':
        'https://download.pytorch.org/models/quantized/resnext101_32x8_fbgemm_09835ccf.pth',
}


class QuantizableBasicBlock(BasicBlock):
    def __init__(self, *args, **kwargs):
        super(QuantizableBasicBlock, self).__init__(*args, **kwargs)
        self.add_relu = torch.nn.quantized.FloatFunctional()

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.add_relu.add_relu(out, identity)

        return out

    def fuse_model(self):
        torch.quantization.fuse_modules(self, [['conv1', 'bn1', 'relu'],
                                               ['conv2', 'bn2']], inplace=True)
        if self.downsample:
            torch.quantization.fuse_modules(self.downsample, ['0', '1'], inplace=True)


class QuantizableBottleneck(Bottleneck):
    def __init__(self, *args, **kwargs):
        super(QuantizableBottleneck, self).__init__(*args, **kwargs)
        self.skip_add_relu = nn.quantized.FloatFunctional()
        self.relu1 = nn.ReLU(inplace=False)
        self.relu2 = nn.ReLU(inplace=False)

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)
        out = self.skip_add_relu.add_relu(out, identity)

        return out

    def fuse_model(self):
        fuse_modules(self, [['conv1', 'bn1', 'relu1'],
                            ['conv2', 'bn2', 'relu2'],
                            ['conv3', 'bn3']], inplace=True)
        if self.downsample:
            torch.quantization.fuse_modules(self.downsample, ['0', '1'], inplace=True)


class QuantizableResNet(ResNet):

    def __init__(self, *args, **kwargs):
        super(QuantizableResNet, self).__init__(*args, **kwargs)

        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        # Ensure scriptability
        # super(QuantizableResNet,self).forward(x)
        # is not scriptable
        x = self._forward_impl(x)
        x = self.dequant(x)
        return x

    def fuse_model(self):
        r"""Fuse conv/bn/relu modules in resnet models
        Fuse conv+bn+relu/ Conv+relu/conv+Bn modules to prepare for quantization.
        Model is modified in place.  Note that this operation does not change numerics
        and the model after modification is in floating point
        """

        fuse_modules(self, ['conv1', 'bn1', 'relu'], inplace=True)
        for m in self.modules():
            if type(m) == QuantizableBottleneck or type(m) == QuantizableBasicBlock:
                m.fuse_model()


def _resnet(arch, block, layers, pretrained, progress, quantize, **kwargs):
    model = QuantizableResNet(block, layers, **kwargs)
    _replace_relu(model)
    if quantize:
        # TODO use pretrained as a string to specify the backend
        backend = 'fbgemm'
        quantize_model(model, backend)
    else:
        assert pretrained in [True, False]

    if pretrained:
        if quantize:
            model_url = quant_model_urls[arch + '_' + backend]
        else:
            model_url = model_urls[arch]

        state_dict = load_state_dict_from_url(model_url,
                                              progress=progress)

        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained=False, progress=True, quantize=False, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
        quantize (bool): If True, return a quantized version of the model
    """
    return _resnet('resnet18', QuantizableBasicBlock, [2, 2, 2, 2], pretrained, progress,
                   quantize, **kwargs)


def resnet50(pretrained=False, progress=True, quantize=False, **kwargs):
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
        quantize (bool): If True, return a quantized version of the model
    """
    return _resnet('resnet50', QuantizableBottleneck, [3, 4, 6, 3], pretrained, progress,
                   quantize, **kwargs)

In [ ]:
from fastai.vision.all import *
BS = 16
#         .label_from_df(cols='is_healthy', label_cls=FloatList)
#         .transform(tfms,size=SZ) #Data augmentation
#         .databunch(bs=BS)
#         .normalize(imagenet_stats)
dls = ImageDataLoaders.from_df(df=df2, path=Path('../input/cassava-leaf-disease-classification/train_images/'), cols='image', valid_pct=0.3, label_col=1, batch_tfms=Normalize.from_stats(*imagenet_stats), image_tfms = tfms, bs=BS, val_bs=16)
# dsets = db.datasets('../input/cassava-leaf-disease-classification/train_images/')
# dls = db.dataloaders(df)


In [ ]:
from fastai.vision.all import *
#         .label_from_df(cols='is_healthy', label_cls=FloatList)
#         .transform(tfms,size=SZ) #Data augmentation
#         .databunch(bs=BS)
#         .normalize(imagenet_stats)
db = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    
    get_x=lambda o:f'../input/cassava-leaf-disease-classification/train_images/'+o.image_id,
    get_y=lambda o:o.is_healthy)
# dsets = db.datasets('../input/cassava-leaf-disease-classification/train_images/')
batch = db.dataloaders(df)

In [ ]:
dls.show_batch()

In [ ]:
from fastai.losses import CrossEntropyLossFlat
# md_ef =  EfficientNet.from_pretrained('efficientnet-b4', num_classes=1)
learn = Learner(dls, resnet18(pretrained=True), metrics = [accuracy, error_rate])

In [ ]:
lr = 1e-3
learn.fit_one_cycle(3, lr)

In [ ]:
# torch.save(learn.model.state_dict(),'/kaggle/working/verynice18.pth')
torch.save(learn.model.state_dict(),'/kaggle/working/verynice18Legal2.pth')

In [ ]:
def print_model_size(model):
    """ Print the size of the model.
    
    Args:
        model: model whose size needs to be determined

    """
    torch.save(model.state_dict(), "temp.p")
    print('Size of the model(MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [ ]:
import torchvision
# torch.quantization.fuse_modules(learn.model, [['conv1', 'bn1', 'relu'],
#                                                ['conv2', 'bn2']], inplace=True)
model = resnet18(pretrained=True)
model.load_state_dict(torch.load('/kaggle/working/verynice18Legal2.pth'))
print_model_size(model)
backend = "qnnpack"

model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend
model_static_quantized = torch.quantization.prepare(model, inplace=False)
model_static_quantized = torch.quantization.convert(model_static_quantized, inplace=False)
print_model_size(model_static_quantized)
# model_static_quantized.eval()

In [ ]:
# Everything Past this point is experimental code

# Everything Past this point is experimental code

In [ ]:
learn.fine_tune(30)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
learn2 = cnn_learner(dls, resnet18, metrics = [accuracy, error_rate], model_dir="models").to_fp16()
learn2.fine_tune(1)

In [ ]:
?? fastai.vision.models.resnet18

In [ ]:
import dill
modeltosave=learn2.model
modeltosave.cpu()
model = torch.load('/kaggle/working/resnet50.pth')
torch.save(model.state_dict(),'/kaggle/working/resnet50.pkl')
torch.save(modeltosave, '/kaggle/working/resnet18.pkl', pickle_module=dill)

In [ ]:
def print_model_size(model):
    """ Print the size of the model.
    
    Args:
        model: model whose size needs to be determined

    """
    torch.save(model.state_dict(), "temp.p")
    print('Size of the model(MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [ ]:
model = torch.load('/kaggle/working/resnet18.pkl')
print_size_of_model(model)
model.to('cpu')
model.eval()
    # Testing with qauntization if quantize=True
modules_to_fuse = [['conv1', 'bn1'],
           ['layer1.0.conv1', 'layer1.0.bn1'],
           ['layer1.0.conv2', 'layer1.0.bn2'],
           ['layer1.1.conv1', 'layer1.1.bn1'],
           ['layer1.1.conv2', 'layer1.1.bn2'],
           ['layer2.0.conv1', 'layer2.0.bn1'],
           ['layer2.0.conv2', 'layer2.0.bn2'],
           ['layer2.0.downsample.0', 'layer2.0.downsample.1'],
           ['layer2.1.conv1', 'layer2.1.bn1'],
           ['layer2.1.conv2', 'layer2.1.bn2'],
           ['layer3.0.conv1', 'layer3.0.bn1'],
           ['layer3.0.conv2', 'layer3.0.bn2'],
           ['layer3.0.downsample.0', 'layer3.0.downsample.1'],
           ['layer3.1.conv1', 'layer3.1.bn1'],
           ['layer3.1.conv2', 'layer3.1.bn2'],
           ['layer4.0.conv1', 'layer4.0.bn1'],
           ['layer4.0.conv2', 'layer4.0.bn2'],
           ['layer4.0.downsample.0', 'layer4.0.downsample.1'],
           ['layer4.1.conv1', 'layer4.1.bn1'],
           ['layer4.1.conv2', 'layer4.1.bn2']]
model = torch.quantization.fuse_modules(model, modules_to_fuse)
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

torch.quantization.prepare(model, inplace=True)
model.eval()
with torch.no_grad():
    for data, target in train_loader:
        model(data)
torch.quantization.convert(model, inplace=True)

print_size_of_model(model)

In [ ]:
model.eval()

In [ ]:
model.state_dict

In [ ]:
learn2.export("/kaggle/working/resnet18.pt")

In [ ]:
import os
os.getcwd()
os.listdir()

In [ ]:
!pip install nvidia-pyindex
!nvidia-index install pytorch-quantization

In [ ]:
print_model_size(resnet18(pretrained=True))

In [ ]:
model = torchvision.models.resnet50(pretrained=True)
model_un = torchvision.models.resnet50(pretrained=False).load_state_dict(torch.load('/kaggle/working/resnet50.pkl'))
torch.quantization.fuse_modules(model_un, [['conv1', 'bn1', 'relu'],
                                               ['conv2', 'bn2']], inplace=True)
print_model_size(model)
backend = "qnnpack"
model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend
model_static_quantized = torch.quantization.prepare(model, inplace=False)
model_static_quantized = torch.quantization.convert(model_static_quantized, inplace=False)
print_model_size(model_static_quantized)

In [ ]:
print(model.state_dict)
print(model_un.state_dict)


In [ ]:
import torchvision
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
model = torchvision.models.resnet50(pretrained=True)
model_un = torch.load('/kaggle/working/resnet50.pth')

model.state_dict = model_un.state_dict
# q_model = copy.deepcopy(model).to('cpu')

print_model_size(model_un)
backend = "qnnpack"
model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend
model_static_quantized = torch.quantization.prepare(model_un, inplace=False)
model_static_quantized = torch.quantization.convert(model_static_quantized, inplace=False)
print_model_size(model_static_quantized)

In [ ]:
from torch.quantization import QuantStub, DeQuantStub

def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


class ConvBNReLU(nn.Sequential):
    def __init__(self, in_planes, out_planes, kernel_size=3, stride=1, groups=1):
        padding = (kernel_size - 1) // 2
        super(ConvBNReLU, self).__init__(
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding, groups=groups, bias=False),
            nn.BatchNorm2d(out_planes, momentum=0.1),
            # Replace with ReLU
            nn.ReLU(inplace=False)
        )


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = int(round(inp * expand_ratio))
        self.use_res_connect = self.stride == 1 and inp == oup

        layers = []
        if expand_ratio != 1:
            # pw
            layers.append(ConvBNReLU(inp, hidden_dim, kernel_size=1))
        layers.extend([
            # dw
            ConvBNReLU(hidden_dim, hidden_dim, stride=stride, groups=hidden_dim),
            # pw-linear
            nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
            nn.BatchNorm2d(oup, momentum=0.1),
        ])
        self.conv = nn.Sequential(*layers)
        # Replace torch.add with floatfunctional
        self.skip_add = nn.quantized.FloatFunctional()

    def forward(self, x):
        if self.use_res_connect:
            return self.skip_add.add(x, self.conv(x))
        else:
            return self.conv(x)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn2)
interp.plot_confusion_matrix()

In [ ]:
learn2.recorder.plot_loss()

In [ ]:
dls = ImageDataLoaders.from_df(df=df2, path=Path('../input/cassava-leaf-disease-classification/train_images/'), cols='image', valid_pct=0.3, label_col=1, batch_tfms=Normalize.from_stats(*imagenet_stats), bs=BS, val_bs=16)

learn2 = cnn_learner(dls, resnet50, metrics = [accuracy, error_rate], model_dir="models").to_fp16()
learn2.fine_tune(5)
learn2.fine_tune(5)
learn2.fine_tune(5)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn2)
interp.plot_confusion_matrix()